In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import random, string
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re
sns.set(context='paper', style='whitegrid', color_codes=True, font_scale=1.8)
colorcycle = [(0.498, 0.788, 0.498),
              (0.745, 0.682, 0.831),
              (0.992, 0.753, 0.525),
              (0.220, 0.424, 0.690),
              (0.749, 0.357, 0.090),
              (1.000, 1.000, 0.600),
              (0.941, 0.008, 0.498),
              (0.400, 0.400, 0.400)]
sns.set_palette(colorcycle)
mpl.rcParams['figure.max_open_warning'] = 65
mpl.rcParams['figure.figsize'] = [12, 7]

from speclib import misc, plotting, loaders

%matplotlib inline 

# Resample

http://pandas.pydata.org/pandas-docs/stable/timeseries.html#resampling

In [2]:
rng = pd.date_range('1/1/2012', periods=100, freq='S')
rng[:8]

DatetimeIndex(['2012-01-01 00:00:00', '2012-01-01 00:00:01',
               '2012-01-01 00:00:02', '2012-01-01 00:00:03',
               '2012-01-01 00:00:04', '2012-01-01 00:00:05',
               '2012-01-01 00:00:06', '2012-01-01 00:00:07'],
              dtype='datetime64[ns]', freq='S')

In [3]:
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng) 

In [4]:
ts.resample('5Min').sum() 

2012-01-01    23113
Freq: 5T, dtype: int64

In [5]:
ts.resample('5Min').mean() 

2012-01-01    231.13
Freq: 5T, dtype: float64

In [6]:
ts.resample('20S').mean()

2012-01-01 00:00:00    213.10
2012-01-01 00:00:20    202.70
2012-01-01 00:00:40    256.40
2012-01-01 00:01:00    220.70
2012-01-01 00:01:20    262.75
Freq: 20S, dtype: float64

In [7]:
rng2 = np.random.permutation(pd.date_range('1/1/2012', periods=1000, freq='S'))[:350]
ts2 = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)

In [8]:
ts2.resample('2Min').sum() 

2012-01-01    25493
Freq: 2T, dtype: int64

In [9]:
ts2.resample('2Min').count() 

2012-01-01    100
Freq: 2T, dtype: int64

In [10]:
ts.resample("5Min", closed='right').mean() 

2011-12-31 23:55:00    256.000000
2012-01-01 00:00:00    230.878788
Freq: 5T, dtype: float64

In [11]:
ts.resample("5Min", closed='left').mean()

2012-01-01    231.13
Freq: 5T, dtype: float64

In [12]:
ser_1 = pd.Series([random.choice(string.ascii_uppercase) for _ in range(len(rng2))]) 
ser_2 = pd.Series(np.random.randint(0, 10, len(rng2)))
ser_3 = pd.Series([random.choice(string.ascii_uppercase) for _ in range(len(rng2))]) 
df2 = pd.DataFrame(np.array([ser_1, ser_2, ser_3]).T, index=rng2, columns=['user', 'value', 'adjacent'])

In [13]:
df2.head(12)

,user,value,adjacent
2012-01-01 00:10:43,S,8,M
2012-01-01 00:03:27,L,1,U
2012-01-01 00:11:02,N,4,D
2012-01-01 00:00:58,G,5,C
2012-01-01 00:16:28,E,9,O
2012-01-01 00:05:54,S,5,Y
2012-01-01 00:03:50,D,9,M
2012-01-01 00:08:05,I,3,A
2012-01-01 00:16:24,A,2,E
2012-01-01 00:15:13,Z,2,O


In [14]:
%timeit df2[['user', 'adjacent']].groupby([pd.Grouper(freq="6Min"), 'user']).count() 
%timeit pd.DataFrame(df2.groupby([pd.Grouper(freq="6Min"), 'user']).adjacent.count())

100 loops, best of 3: 7.92 ms per loop
100 loops, best of 3: 6.18 ms per loop


In [15]:
df2.groupby(['user', pd.Grouper(freq="6Min")]).adjacent.agg(["count", "sum"]) 

count        sum
user                                      
A    2012-01-01 00:00:00      4       XHIM
     2012-01-01 00:06:00      6     KDCCKS
     2012-01-01 00:12:00      5      EROAD
B    2012-01-01 00:00:00      7    XRTPQPC
     2012-01-01 00:06:00      2         VF
     2012-01-01 00:12:00      4       WEJC
C    2012-01-01 00:00:00      3        NPM
     2012-01-01 00:06:00      9  ZEQUQVEIK
     2012-01-01 00:12:00      5      YMFGV
D    2012-01-01 00:00:00      7    MHRFKQN
     2012-01-01 00:06:00      3        LNS
     2012-01-01 00:12:00      4       LFEN
E    2012-01-01 00:00:00      2         TY
     2012-01-01 00:06:00      1          Q
     2012-01-01 00:12:00      4       OTEP
F    2012-01-01 00:00:00      7    QVKPPNB
     2012-01-01 00:06:00      5      IEYUD
     2012-01-01 00:12:00      6     XMXOCM
G    2012-01-01 00:00:00      6     CNLTCG
     2012-01-01 00:06:00      2         BM
     2012-01-01 00:12:00      4       HEOS
H    2012-01-01 00:00:00      4       ZIHA
     2012-01-01 00:06:00      1          V
     2012-01-01 00:12:00      2         FQ
I    2012-01-01 00:00:00      2         TI
     2012-01-01 00:06:00      8   AQXQSYAI
     2012-01-01 00:12:00      3        CKW
J    2012-01-01 00:00:00      4       QSUA
     2012-01-01 00:06:00      9  IBQPRYXXZ
     2012-01-01 00:12:00      5      IRUKL
...                         ...        ...
Q    2012-01-01 00:00:00      3        BLO
     2012-01-01 00:06:00      7    OGAAOCV
     2012-01-01 00:12:00      3        MJL
R    2012-01-01 00:00:00      4       RJDO
     2012-01-01 00:06:00      3        BGY
     2012-01-01 00:12:00      4       GBYW
S    2012-01-01 00:00:00      5      YCLZT
     2012-01-01 00:06:00      9  MIDBPBPPX
     2012-01-01 00:12:00      5      MFVRY
T    2012-01-01 00:00:00      5      RJOOM
     2012-01-01 00:06:00      3        ZGG
     2012-01-01 00:12:00      4       FXAZ
U    2012-01-01 00:00:00      1          L
     2012-01-01 00:06:00      5      JFMVL
     2012-01-01 00:12:00      2         IB
V    2012-01-01 00:00:00      4       FFBJ
     2012-01-01 00:06:00      7    EGDRGVW
     2012-01-01 00:12:00      3        PDI
W    2012-01-01 00:00:00      3        ZTX
     2012-01-01 00:06:00      7    QGCJEDP
     2012-01-01 00:12:00      5      ZGIKL
X    2012-01-01 00:00:00      6     MQJGVR
     2012-01-01 00:06:00      6     UEPIRJ
     2012-01-01 00:12:00      4       PDUL
Y    2012-01-01 00:00:00      7    OXJDXGK
     2012-01-01 00:06:00      4       WPGS
     2012-01-01 00:12:00      3        JMI
Z    2012-01-01 00:00:00      2         RB
     2012-01-01 00:06:00      5      KOUYY
     2012-01-01 00:12:00      5      OOZOU

[77 rows x 2 columns]

In [16]:
df2.groupby(['user', df2.index.weekday_name]).adjacent.agg(["count", "sum"])

,,count,sum
user,,,
A,Sunday,15,EROXKADDCHCIKSM
B,Sunday,13,XWEVJRTPQPCFC
C,Sunday,17,ZYNEQUMQVFGEPIVMK
D,Sunday,14,MLLHRFFEKNQSNN
E,Sunday,7,OTEQTPY
F,Sunday,18,QIVXMKPXPEYUNOCBDM
G,Sunday,12,CHENOLBSTCGM
H,Sunday,7,ZVIHFAQ
I,Sunday,13,AQCTXKWIQSYAI


In [17]:
tmp = pd.DataFrame(pd.date_range('1/1/2012', '1/7/2012', freq='D'))
tmp['weekname'] = tmp[0].dt.weekday_name
tmp['weekcode'] = tmp[0].dt.weekday
tmp.sort_values('weekcode')

,0,weekname,weekcode
1,2012-01-02,Monday,0
2,2012-01-03,Tuesday,1
3,2012-01-04,Wednesday,2
4,2012-01-05,Thursday,3
5,2012-01-06,Friday,4
6,2012-01-07,Saturday,5
0,2012-01-01,Sunday,6


In [18]:
rng3 = np.random.permutation(pd.date_range('1/1/2012', periods=100_000, freq='Min'))[:15000]
ts3 = pd.Series(np.random.randint(0, 500, len(rng3)), index=rng3)
ser_1 = pd.Series([random.choice(string.ascii_letters) for _ in range(len(rng3))]) 
ser_2 = pd.Series(np.random.randint(0, 10, len(rng3)))
ser_3 = pd.Series([random.choice(string.ascii_letters) for _ in range(len(rng3))]) 
df3 = pd.DataFrame(np.array([ser_1, ser_2, ser_3]).T, index=rng3, columns=['user', 'value', 'adjacent'])
df3.head(8)

,user,value,adjacent
2012-01-28 01:17:00,Z,5,i
2012-02-21 15:12:00,w,7,l
2012-01-07 07:07:00,x,1,H
2012-02-24 08:03:00,x,9,U
2012-02-12 08:59:00,R,4,g
2012-03-08 14:20:00,U,0,r
2012-03-04 06:20:00,q,3,c
2012-03-09 14:19:00,C,4,g


In [19]:
df3['before_workday'] = pd.Series(df3.index.weekday, index=df3.index).isin({0, 1, 2, 3, 6})

df3.sample(7) 

,user,value,adjacent,before_workday
2012-01-19 21:18:00,R,6,f,True
2012-02-11 12:13:00,x,2,L,False
2012-01-22 11:34:00,F,4,R,True
2012-01-20 19:38:00,J,2,r,False
2012-03-07 04:25:00,W,2,S,True
2012-01-22 12:32:00,R,9,O,True
2012-02-18 21:28:00,x,3,E,False


In [20]:
df3['free_time'] = (16 < df3.index.hour) | (df3.index.hour < 7)

df3.head(12)

,user,value,adjacent,before_workday,free_time
2012-01-28 01:17:00,Z,5,i,False,True
2012-02-21 15:12:00,w,7,l,True,False
2012-01-07 07:07:00,x,1,H,False,False
2012-02-24 08:03:00,x,9,U,False,False
2012-02-12 08:59:00,R,4,g,True,False
2012-03-08 14:20:00,U,0,r,True,False
2012-03-04 06:20:00,q,3,c,True,True
2012-03-09 14:19:00,C,4,g,False,False
2012-01-02 12:42:00,J,8,V,True,False
2012-01-16 10:46:00,K,6,q,True,False


In [21]:
df4 = df3[df3.before_workday & df3.free_time][['user', 'value', 'adjacent']]
df4.head(12)

,user,value,adjacent
2012-03-04 06:20:00,q,3,c
2012-02-01 22:28:00,h,8,t
2012-02-27 06:44:00,e,3,c
2012-01-11 18:32:00,G,1,H
2012-02-19 05:20:00,a,4,a
2012-03-01 19:11:00,E,8,c
2012-01-23 03:41:00,U,3,y
2012-02-19 04:26:00,y,6,N
2012-01-09 20:30:00,x,0,q
2012-01-12 04:51:00,F,2,r


In [22]:
df4.groupby(['user', df4.index.weekday]).adjacent.agg(["count", "sum"]) 

count                                 sum
user                                             
A    0     24            OOnFQCVSWWYWrQYZWrkIcmiI
     1     33   OGLbhqnEChFkaYNyJmieCnMbCtHwZgyfO
     2     27         FWIMpVckVXGIHGLyKidcMvRWzRw
     3     34  RSCNUgHzTOGSPcqXZCYLRfHPXEKUMBMWON
     6     18                  RCNcclUKCheVVMWXSY
B    0     28        CuNyxiCkugaXBHqIjVngTQzGCpns
     1     25           buUeKJDEqFUEdecklMekfypeg
     2     31     YhRaqAsqzUZZiYXaCGPZrduzQesVsKa
     3     31     pCwvqwPKOCaJOMQInjorefltZDIrCRo
     6     29       LSsXvWcGWJcWpdNpWcDcBrPsXLkko
C    0     27         tmtbZEezKNsqRiiZyUvTQwqwbpA
     1     19                 zRgqvUCMNuaMzxNnWiG
     2     15                     TcCiCMjumOAjEEu
     3     32    NAGCsxoqMDpmfmuvicSfYCmkbyJSaASr
     6     23             SaSKNRmhZlmwxpttwnBKHSV
D    0     16                    RPuYOhltSAfUeYvM
     1     22              aquOeiheYlQzEXaYjfnArw
     2     22              OQVEIHlKsapNNGZIWboypC
     3     23             uEHxyHmzHNWyddqxaezdbMo
     6     25           iGpwUoMIVgLWPisHCHNQRngoF
E    0     16                    ANgnriYHLjvIMvCn
     1     23             kMeiwqpVIubCaooTeyYKrHG
     2     20                NarpyurDryvSmbKuftyf
     3     22              cvGUMPlfgCEIpljjsDdaEL
     6     27         HUzgbBqakYLMCnvvXEKhdVyBDmo
F    0     22              BrgAmcDQQzMgzxlnTExkEZ
     1     21               tSMVCKRqJucPbfEwtuDOP
     2     20                tJfXwBVzSxsYaIvawekZ
     3     20                rFWBVGSPKGVRBFQXkzxG
     6     24            eCQqTigGUkplfewTxKQQaYjS
...       ...                                 ...
u    0     27         toEcdhPinMwVvUYnATWsxyvlGNq
     1     25           YLdGaVwRbcuumiOmBZltWygwR
     2     28        mfGPOHpHezBtZXdfffeOemkFJkQE
     3     27         zhKbtBpdXAJKbyMHkzTFxjfUmod
     6     25           SDFecLLhMQCuWBsYuaAeWpaaF
v    0     22              kRabBmbxVLxIQjJnsZQkOX
     1     33   EMgfVLhhOBBYJtbxVMZDTteNCRrQIymPm
     2     26          GqGfwtwiUEbeDJQptHvTrWCmwL
     3     28        KlzIsfxMgDtAJMJrrhCcjyEnTnnQ
     6     29       NmqFeDLLXHKRrwhuWbMcSLlOQlmnL
w    0     25           uBMtbidhvPxLwfBkRWDGOYeux
     1     29       ZfTyuKGkeVjsAukEXurVWsvewkhzV
     2     22              XtqPPKfIQvQMtrIEoUQSxO
     3     25           afdLRfcXwNLAJRlcBcgEXbBed
     6     20                wEaNMFvmpyMJSCbaobhH
x    0     23             qoVVHZzrfHjmUBFfcankUon
     1     18                  mgyfOpGdbGACHHwoGZ
     2     30      HRRBpVRyGzthmRskCwnZnvMuIKTWdN
     3     23             AflqovHzSOuHjoXJLaZgmoS
     6     16                    ZSTDaFbRITIawLCQ
y    0     28        xnHxMmyMlZDeoernvvoXJZUWOAbU
     1     22              fwuXtxQPnHpasVutBQrTlw
     2     22              yOsvjwditDMxcNVvTZFMEU
     3     23             cMEbPsnPhciMLJqxxLPUxrF
     6     23             NdNOEQpWpFCbrcsWUCJPgsk
z    0     24            DJHtkzyIfCGPXHxnRgnWplsa
     1     32    LbKaKXjylAcXBaSKloOHGPtcOJiiLqhB
     2     26          VHvCGkAejcHeMEcpHaBZEUNrdO
     3     21               ngQhVMLhoDDwGRnHjuNyf
     6     27         bhXzHkgYWQLFvaycwMONPMboOWr

[260 rows x 2 columns]

In [23]:
tmp = pd.Series([1, 2, 3, np.NaN])
tmp

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [24]:
tmp.count() 

3

In [25]:
tmp.size

4

In [26]:
minutes = df2.index.minute 

In [27]:
%timeit (df2.index.minute > 3) & (df2.index.minute < 8)
%timeit (minutes > 3) & (minutes < 8)

The slowest run took 7.03 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 91.9 µs per loop
The slowest run took 17.99 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.31 µs per loop


In [28]:
df5 = df3.reset_index().set_index('user').rename(columns={'index': 'time'})

In [29]:
df5

,time,value,adjacent,before_workday,free_time
user,,,,,
Z,2012-01-28 01:17:00,5,i,False,True
w,2012-02-21 15:12:00,7,l,True,False
x,2012-01-07 07:07:00,1,H,False,False
x,2012-02-24 08:03:00,9,U,False,False
R,2012-02-12 08:59:00,4,g,True,False
U,2012-03-08 14:20:00,0,r,True,False
q,2012-03-04 06:20:00,3,c,True,True
C,2012-03-09 14:19:00,4,g,False,False
J,2012-01-02 12:42:00,8,V,True,False
